# Functional Closures and Decorators

##  Closures
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing
  * G = global
  * B = builtin (e.g., len() function)

In [1]:
global_var = 'global var'
   
def outer_func():
    local_var = 'local to outer_func()'
    def inner_func():
        inner_var = 'local to inner_func()'
        print(local_var, 'enclosing scope')
        print(global_var, 'global scope')
        
    inner_func()

In [2]:
outer_func()

local to outer_func() enclosing scope
global var global scope


* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which _remembers_ a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [3]:
def make_adder(x):
    # print x's address
    print(f'id(x): 0x{id(x):x}')
    # enclosing scope
    
    def adder(y):
        print('in adder')
        return x + y # Python uses LEGB to find 'x'
    
    print(f'id(adder): 0x{id(adder):x}')
    return adder # we return the function adder (i.e., its address in memory) 

add39 = make_adder(39)
print('about to call add39()')
add39(109)

id(x): 0x7fd58c4f5448
id(adder): 0x7fd5676104a0
about to call add39()
in adder


148

In [4]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
print(type(add39), repr(add39), sep='\n')

<class 'function'>
<function make_adder.<locals>.adder at 0x7fd5676104a0>


In [5]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x7fd584135f60: int object at 0x7fd58c4f5448>,)

In [6]:
# notice that the cell object has a reference to an int object
add39.__closure__[0].cell_contents

39

In [7]:
print(make_adder.__closure__)

None


* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [8]:
def logging(f):
    def wrapper(*args, **kwargs):
        print(f'Calling {f}({args}, {kwargs})')
        return f(*args, **kwargs)
    return wrapper

In [9]:
logging_add39 = logging(add39)

In [10]:
print(add39(5)) # remember that add39 just adds 39 to our argument

in adder
44


In [11]:
print(logging_add39(5))

Calling <function make_adder.<locals>.adder at 0x7fd5676104a0>((5,), {})
in adder
44


In [12]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder(y)>

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [13]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

In [14]:
def add_things(a, b):
    return a + b

print('Running plain old add_things()')
print(add_things(13, 5))

Running plain old add_things()
18


In [15]:
# manual decorator assignment
add_things = document_it(add_things) 

In [16]:
#print('Running cooler_add_things()')
add_things(13, 5)

Running function: add_things
Positional arguments: (13, 5)
Keyword arguments: {}
Result: 18


18

In [17]:
# decorator shorthand for what we did above

# from your_decorators import document_it

@document_it
def add_things(a, b):
    return a + b

# add_things = document_it(add_things)

print(add_things(13, -5))

Running function: add_things
Positional arguments: (13, -5)
Keyword arguments: {}
Result: 8
8


## Lab: Decorators
1. Create some function which takes an integer as its parameter
  * create a wrapper that ensures the parameter is positive
  * use that wrapper to decorate your original function
2. Make a timer decorator that computes the elapsed time of the function wrapped by it


In [20]:
from functools import wraps

# Decorator to ensure the parameter is positive
def ensure_positive(func):
    @wraps(func)
    def wrapper(n):
        if n < 0:
            print(f"Warning: Negative input {n} converted to {abs(n)}")
            n = abs(n)  # Convert to positive
        return func(n)
    return wrapper

# Function to compute factorial (example)
@ensure_positive
def factorial(n):
    """Computes factorial of a number (non-recursive)."""
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

# Example usage:
print(factorial(-5))  # Output: Warning message & factorial of 5


120


In [22]:
import time

# Timer decorator
def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()  # Record start time
        result = func(*args, **kwargs)
        end_time = time.perf_counter()  # Record end time
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' took {elapsed_time:.6f} seconds")
        return result
    return wrapper

# Example usage with factorial function
@timer
def slow_factorial(n):
    """Computes factorial with a delay to demonstrate timing."""
    time.sleep(1)  # Simulating a slow function
    return factorial(n)

# Call the decorated function
print(slow_factorial(5))


Function 'slow_factorial' took 1.000086 seconds
120
